In [44]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

In [45]:
import tensorflow as tf

In [46]:
import sparknlp
spark = sparknlp.start()

In [47]:
# pip install ipywidgets==7.4.2
import transformers
from transformers import TFRobertaModel, TFAutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import tensorflow
tokenizer = AutoTokenizer.from_pretrained("Woonn/bert-base-finetuned-emotion")

model = AutoModelForSequenceClassification.from_pretrained("Woonn/bert-base-finetuned-emotion")

In [55]:
MODEL_NAME_TF='bert_tf' # Change me
tokenizer.save_pretrained(f'./{MODEL_NAME_TF}_tokenizer/')
model.save_pretrained(f'./{MODEL_NAME_TF}', saved_model=True, save_format='tf')

In [54]:
model_tf = TFAutoModel.from_pretrained("Woonn/bert-base-finetuned-emotion", from_pt=True)
model_tf.save_pretrained(f'./{MODEL_NAME_TF}', saved_model=True, save_format='tf')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['classifier.weight', 'bert.embeddings.position_ids', 'classifier.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


INFO:tensorflow:Assets written to: ./bert_tf\saved_model\1\assets


INFO:tensorflow:Assets written to: ./bert_tf\saved_model\1\assets


In [52]:
# Copy the weights from the PyTorch model to the TensorFlow model
for i, layer in enumerate(model.encoder.layer):
    model_tf.get_layer(index=i).set_weights([
        layer.attention.self.query.weight.cpu().numpy().T,
        layer.attention.self.query.bias.cpu().numpy(),
        layer.attention.self.key.weight.cpu().numpy().T,
        layer.attention.self.key.bias.cpu().numpy(),
        layer.attention.self.value.weight.cpu().numpy().T,
        layer.attention.self.value.bias.cpu().numpy(),
        layer.attention.output.dense.weight.cpu().numpy().T,
        layer.attention.output.dense.bias.cpu().numpy(),
        layer.attention.output.LayerNorm.weight.cpu().numpy(),
        layer.attention.output.LayerNorm.bias.cpu().numpy(),
        layer.intermediate.dense.weight.cpu().numpy().T,
        layer.intermediate.dense.bias.cpu().numpy(),
        layer.output.dense.weight.cpu().numpy().T,
        layer.output.dense.bias.cpu().numpy(),
        layer.output.LayerNorm.weight.cpu().numpy(),
        layer.output.LayerNorm.bias.cpu().numpy(),
    ])

AttributeError: 'BertForSequenceClassification' object has no attribute 'encoder'

In [58]:
loaded_model = TFBertForSequenceClassification.from_pretrained(f'./{MODEL_NAME_TF}')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ./bert_tf and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
import shutil

vocab_pth = f"./{MODEL_NAME_TF}_tokenizer/vocab.txt"
saved_model_pth = f'./{MODEL_NAME_TF}/saved_model/1/assets/'
shutil.move(vocab_pth, saved_model_pth)


'./bert_tf/saved_model/1/assets/vocab.txt'

In [60]:
from pyspark.sql.types import StructType, StructField, StringType

# Define the schema of the DataFrame
schema = StructType([StructField("text", StringType(), True)])

# Load the text data into a DataFrame
df = spark.read.format("csv").schema(schema).option("header", True).option("encoding", "UTF-8").load("data.csv")
df.show()

+------------------------------+
|                          text|
+------------------------------+
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
|블랙핑크 세상에서 제일 예뻐!!!|
+------------------------------+
only showing top 20 rows

